# Importing the data and splitting it into training and test data

In [1]:
import numpy as np
import random
random.seed(43)
#import seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#seaborn.set(style='whitegrid'); seaborn.set_context('talk')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.datasets import load_iris
iris_data = load_iris()


from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
print(iris_data['DESCR'])

In [ ]:
data = np.column_stack((iris_data.data,iris_data.target.T)) #Joins the 4 features and the target variable which is encoded as 0,1,2 for setosa, versicolour and virginica

Since all the categories are one after the other, we need to shuffle it first

In [ ]:
np.random.shuffle(data)

## train-test split and X and Y split

In [ ]:
training, testing=train_test_split(data, test_size=0.2, train_size=0.8, random_state=43, shuffle=True, stratify=None)
train_X = np.array([i[:4] for i in training])
train_y = np.array([i[4] for i in training])
test_X = np.array([i[:4] for i in testing])
test_y = np.array([i[4] for i in testing])

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)


# Multilayer Perceptron Implementation

## Activation function - Hyperbolic tangent

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin #base estimator is the base class for all estimators, classifier mixin returns the mean accuracy on the given test data and labels.

class MultilayerPerceptron(BaseEstimator,ClassifierMixin):
    def __init__(self):
        self.inputLayer = 4                        # number of neurons in Input Layer
        self.hiddenLayer = 5                       # number of neurons in Hidden Layer
        self.outputLayer = 3                       # number of neurons Output Layer
        self.learningRate = 0.001                  # Learning rate
        self.max_epochs = 1000                     # Epochs
        self.BiasHiddenValue = -1                   # Bias HiddenLayer
        self.BiasOutputValue = -1                  # Bias OutputLayer
        self.activation = self.tanh # Activation function
        self.deriv = self.deriv_tanh
        
        
        #defining the starting weights and biases
        self.WEIGHT_hidden = self.starting_weights(self.hiddenLayer, self.inputLayer)
        self.WEIGHT_output = self.starting_weights(self.outputLayer, self.hiddenLayer)
        self.BIAS_hidden = np.array([self.BiasHiddenValue for i in range(self.hiddenLayer)])
        self.BIAS_output = np.array([self.BiasOutputValue for i in range(self.outputLayer)])
        self.classes_number = 3 
        
    
    def tanh(self,x):
        return np.tanh(x)

    def deriv_tanh(self,x):
        return (1-(self.tanh(x)**2))
    
    def starting_weights(self, x, y):
        #return [[ random.uniform(0,0.5)for i in range(x)] for j in range(y)]
        return [[2  * random.random() - 1 for i in range(x)] for j in range(y)]

    
    def backpropagation(self,x):
        #Step 1- find the error in the output layer, update the weights and bias between output layer and hidden layer
        
        DELTA_output = []
        
        ERROR_output = self.output - self.OUTPUT_L2 #expected output- received output
        DELTA_output = ((-1)*(ERROR_output) * self.deriv(self.OUTPUT_L2)) # (error*deriv) why -1?
        
        for i in range(self.hiddenLayer):#loops to connect from each node in hidden layer (i) to each node in output layer (j)
            for j in range(self.outputLayer):
                self.WEIGHT_output[i][j] -= (self.learningRate * (DELTA_output[j] * self.OUTPUT_L1[i])) #(error*deriv(j)*output from prev layer(i))
                self.BIAS_output[j] -= (self.learningRate * DELTA_output[j])
        
        #Step 2- find the error in the hidden layer, update the weights and bias between hidden layer and input layer
        
        delta_hidden = np.matmul(self.WEIGHT_output, DELTA_output)* self.deriv(self.OUTPUT_L1) #formula for delta term for hidden layer= (delta term for output layer multiplied by the weight to the output layer) multiplied by the derivative of the input to the hidden layer 
        
        
        for i in range(self.inputLayer):
            for j in range(self.hiddenLayer):
                self.WEIGHT_hidden[i][j] -= (self.learningRate * (delta_hidden[j] * x[i]))#multiplying the delta term by the input 
                self.BIAS_hidden[j] -= (self.learningRate * delta_hidden[j])
        
    def show_err_graphic(self,v_erro,v_epochs):
        plt.figure(figsize=(9,4))
        plt.plot(v_epochs, v_erro, "m-",color="b", marker=11)
        plt.xlabel("Number of Epochs")
        plt.ylabel("Squared error (MSE) ");
        plt.title("Error Minimization")
        plt.show()
        
        
    def predict(self, X, y):#Returns the predictions for every element of X
        
        predictions = []

        forward = np.matmul(X,self.WEIGHT_hidden) + self.BIAS_hidden#input to hidden layer
        forward = np.matmul(forward, self.WEIGHT_output) + self.BIAS_output#hidden layer to output layer

        for i in forward:
            predictions.append(max(enumerate(i), key=lambda x:x[1])[0])#WHAT DOES THIS DO

        array_score = []# creating output table
        for i in range(len(predictions)):
            if predictions[i] == 0: 
                array_score.append([i, 'Iris-setosa', predictions[i], y[i]])
            elif predictions[i] == 1:
                 array_score.append([i, 'Iris-versicolour', predictions[i], y[i]])
            elif predictions[i] == 2:
                 array_score.append([i, 'Iris-virginica', predictions[i], y[i]])


        dataframe = pd.DataFrame(array_score, columns=['_id', 'class', 'output', 'expected_output'])
        return predictions, dataframe
        
        
    def fit(self, X, y):  
        count_epoch = 1
        total_error = 0
        n = len(X); 
        epoch_array = []
        error_array = []
        W0 = []
        W1 = []
        while(count_epoch <= self.max_epochs):
            for idx,inputs in enumerate(X): 
                #initialize expected input
                self.output = np.zeros(self.classes_number)
                #forward propogation i.e activation((inputs * weights)+bias)
                self.OUTPUT_L1 = self.activation((np.dot(inputs, self.WEIGHT_hidden)) + self.BIAS_hidden.T)
                self.OUTPUT_L2 = self.activation(np.dot(self.OUTPUT_L1, self.WEIGHT_output) + self.BIAS_output.T)
                #giving values for expected input based on the ground truth we have in y                    
                if(y[idx] == 0): 
                    self.output = np.array([1,0,0]) #Class1 {1,0,0}
                elif(y[idx] == 1):
                    self.output = np.array([0,1,0]) #Class2 {0,1,0}
                elif(y[idx] == 2):
                    self.output = np.array([0,0,1]) #Class3 {0,0,1}

                square_error = 0
                for i in range(self.outputLayer):
                    erro=0

                    erro = (self.output[i] - self.OUTPUT_L2[i])**2
                    square_error = (square_error + (0.05 * erro))#1/2 * error squared
                    total_error = total_error + square_error
                    
                #now, backpropogate
                self.backpropagation(inputs)

                total_error = (total_error / n)
                if((count_epoch % 50 == 0)or(count_epoch == 1)):
                    print("Epoch ", count_epoch, "- Total Error: ",total_error)
                    error_array.append(total_error)
                    epoch_array.append(count_epoch)
                    
                W0.append(self.WEIGHT_hidden)
                W1.append(self.WEIGHT_output)

                count_epoch+=1


        self.show_err_graphic(error_array,epoch_array)

    
    

In [ ]:
Perceptron = MultilayerPerceptron()
Perceptron.fit(train_X,train_y)


# Testing our neural network

In [ ]:
pred, dataframe = Perceptron.predict(test_X, test_y)
hits = n_set = n_vers = n_virg = 0
score_set = score_vers = score_virg = 0
for j in range(len(test_y)):
    if(test_y[j] == 0): 
        n_set += 1
    elif(test_y[j] == 1): 
        n_vers += 1
    elif(test_y[j] == 2): 
        n_virg += 1
        
for i in range(len(test_y)):
    if test_y[i] == pred[i]: 
        hits += 1
    if test_y[i] == pred[i] and test_y[i] == 0:
        score_set += 1
    elif test_y[i] == pred[i] and test_y[i] == 1:
        score_vers += 1
    elif test_y[i] == pred[i] and test_y[i] == 2:
        score_virg += 1    
         
hits = (hits / len(test_y)) * 100
faults = 100 - hits

In [ ]:
dataframe

In [ ]:
graph_hits = []
n_samples, n_features = iris_data.data.shape

print("Percents :","%.2f"%(hits),"% hits","and","%.2f"%(faults),"% faults")
print("Total samples of test",n_samples)
print("*Iris-Setosa:",n_set,"samples")
print("*Iris-Versicolour:",n_vers,"samples")
print("*Iris-Virginica:",n_virg,"samples")

graph_hits.append(hits)
graph_hits.append(faults)
labels = 'Hits', 'Faults';
sizes = [96.5, 3.3]
explode = (0, 0.14)

fig1, ax1 = plt.subplots();
ax1.pie(graph_hits,colors=['green','red'],labels=labels, autopct='%1.1f%%',startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
acc_set = (score_set/n_set)*100
acc_vers = (score_vers/n_vers)*100
acc_virg = (score_virg/n_virg)*100
print("- Accuracy Iris-Setosa:","%.2f"%acc_set, "%")
print("- Accuracy Iris-Versicolour:","%.2f"%acc_vers, "%")
print("- Accuracy Iris-Virginica:","%.2f"%acc_virg, "%")
names = ["Setosa","Versicolour","Virginica"]
x1 = [2.0,4.0,6.0]
fig, ax = plt.subplots()
r1 = plt.bar(x1[0], acc_set,color='orange',label='Iris-Setosa')
r2 = plt.bar(x1[1], acc_vers,color='green',label='Iris-Versicolour')
r3 = plt.bar(x1[2], acc_virg,color='purple',label='Iris-Virginica')
plt.ylabel('Scores %')
plt.xticks(x1, names);plt.title('Scores by iris flowers - Multilayer Perceptron')
plt.show()